<a href="https://colab.research.google.com/github/adalbertii/Modele-regresyjne/blob/main/mpg-case-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analiza modeli neuronowych**

Wykorzystanie danych  [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg).

[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).


Demonstracja sposobu budowania modelu przeznaczonego do predykcji efektywnosci spalania paliwa.

Tok działania:
- pobranie danych i ich wstępna analiza
- uproszczona standaryzacja danych

- budowa modelu opratego na jednym neuronie i jednym atrybucie (zmiennej objaśnijającej)

- budowa modelu składającego się z jednego neurona i wykorzystującego wszystkie zmienne objaśniajace

*   budowa modelu składajacego się z 64 neuronów i jednej zmiennej objaśniajacej

*    budowa modelu 64 neronowego ze wsystkimi zmiennymi objaśniajacymi

In [90]:
!pip install -q seaborn

In [91]:
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow as tf

from tensorflow import keras

In [92]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
np.set_printoptions(precision=3, suppress=True)
print(tf.__version__)

### Pobranie danych do uczenia modeli
Wykorzystanie modułu pandas


In [94]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Liczba cylindrow', 'Pojemność skokowa', 'Moc', 'Waga',
                'Przyspieszenie', 'Rok modelu', 'Pochodzenie']


# mpg - miles per gallon
raw_dataset = pd.read_csv(url,
                          names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
# tworzymy kopie zbioru danych
dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset.head(5)

In [ ]:
## sprawdzamy charakterystykę atrybutów pobranego zbioru danych
dataset.info()

Atrybut 'pochodzenie' wymaga uwagi. To jest ukryta zmienna kategoryczna

In [ ]:
# sprawdzamy rozkład statystyczne zmiennych
dataset.describe().T

In [ ]:
# sprawdzamy korelacje zmiennych
dataset.corr()['MPG'].sort_values(ascending=False)


In [100]:
import plotly.express as px

In [ ]:
#sprawdzamy rozkłady wartości MPG dla różnych krajów pochodzenia
px.histogram(dataset, x='MPG', width=1000, height=400, nbins=50, facet_col='Pochodzenie')

In [ ]:
#sprawdzamy rozkłady wartości MPG w  zależności od liczny cylindów
px.histogram(dataset, x='MPG', width=1000, height=400, nbins=50, facet_col='Liczba cylindrow')

In [ ]:
#sprawdzamy rozkłady wartości 'Rok modelu'
px.histogram(dataset, x='Rok modelu', width=1000, height=400, nbins=50)

### Wstępne przygotowanie danych




In [ ]:
# sprawdzamy czy występują braki danych
dataset.isna().sum()

Istnieją braki danych.
Dla uproszczenia usuwamy rekordy zawierajace braki danych

In [105]:
# Istnieją braki danych. Dla uproszczenia usuwamy rekordy zawierajace braki danych

dataset = dataset.dropna()

In [ ]:
dataset.isna().sum()

In [ ]:
# sprawdzenie czy występują duplikaty
dataset[dataset.duplicated()]

Kolumna 'Pochodzenie` jet atrybutem kategorycznym, nie numerycznym .
Nalezy zatem przeprowadić na tej kolumnie transfomację "one-hot-encod".
W tym celu wykorzystana zostanie  metoda [pd.get_dummies]

In [108]:
# najpierw dokonujemy konwersji liczb (int64)  na oznaczenie tekstowe
dataset['Pochodzenie'] = dataset['Pochodzenie'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset.head(5)

In [ ]:
dataset.Pochodzenie.unique()

In [111]:
# bardzo ważna operacja kodowania zmiennej kategorycznej 'Pochodzenie'
dataset = pd.get_dummies(dataset,  columns=['Pochodzenie'], prefix='',prefix_sep='') ###

In [ ]:
dataset.head(5)

### Podział danych na dane treningowe i testowe

Dane testowe zostaną wykorzystane do końcowej ewaluacji modelu

In [ ]:
dataset.shape

In [114]:
# operacja zwraca 80% danych wybranych losowo)
train_dataset = dataset.sample(frac=0.8, random_state=0)

In [115]:
# usuwamy rekordy o indexach przydzilonych do rekordów z poprzedniej operacji
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_dataset.shape

In [ ]:
test_dataset.shape

### Wstępna analiza danych



In [ ]:
sns.pairplot(train_dataset[['MPG', 'Liczba cylindrow', 'Pojemność skokowa', 'Waga','Moc']], diag_kind='kde')

Weryfikacja rozkładów statystycznych zmiennych objaśniajacych

In [ ]:
train_dataset.describe().transpose()

### Rozdzielenie zmiennych objaśniajacych od zmiennej objaśnianej (etykiety)


In [119]:
# najpierw same cechy (zmienne objaśniajaće)
train_features = train_dataset.copy()
test_features = test_dataset.copy()

In [ ]:
train_features.head(5)

In [121]:
# teraz tzw. etykiety, czyli zmenne objaśniane
# operacja poniższa zwraca warosci kolumny 'MPG' , jedocześnie usuwając tąkolumnę z ramki danych
train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

In [ ]:
train_features.shape

In [ ]:
train_features

In [ ]:
train_labels

## Normalizacja danych



In [ ]:
# najpierw sprawdzam jak bardzo zróznicowane sa  wartościowo poszczególnych atrybutow
train_dataset.describe().transpose()[['mean', 'std']]


Wartość średnia i odchylenie standardowe jest mocno żróżnicowane.

Dobrą praktyką jest normalizacja cech, które używają różnych skal i zakresów.

Jednym z powodów, dla których jest to ważne, jest to, że cechy są mnożone przez wagi modelu. Tak więc skala wyników i skala gradientów zależy od skali danych wejściowych.

Chociaż model *może* uczyć  się bez normalizacji cech, normalizacja sprawia, że szkolenie jest znacznie bardziej stabilne.


### Warstwa normalizująca  dane

Tworzymy instancję normalizatora

In [126]:
normalizer = tf.keras.layers.Normalization(axis=-1)

Dokonujemy normalizacji zmiennych objaśniajacych :

In [127]:
# metoda "adapt" wymaga obiektu np.array
normalizer.adapt(np.array(train_features))

Gdy warstwa jest wywoływana, zwraca dane wejściowe, z każdą cechą niezależnie znormalizowaną:

In [ ]:
# to jest tylko symulacja procesu normalizacji
# demonstracja działania normalizotora
# pobieramy 1 wiersz
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('Wiersz przed normalizacja:', first)
  print()
  print('Wiersz znormalizowany    :', normalizer(first).numpy())

## Regresja liniowa

Przed zbudowaniem głebokiej sieci neuronowej przeanalizujemu budowę liniowej sieci z jedną i wieloma cechami (zmiennymi objaśniajacymi)

### Regresja lionowa z jedną zmienną objaśniajacą

Predykcja 'MPG' na podstwie zmiennej 'Moc'.

Sieć neuronowa będzie zbudowana z dwóch warstw:

- warstwy normalizującej atrybut wejściowy 'moc` (przy użyciu klasy  `tf.keras.layers.Normalization).
- warstwy Dense

In [ ]:
horsepower = np.array(train_features['Moc'])
horsepower
# te dane nie sa jeszcze wyskalowane. Zrobi to odpowiednia warstwa sieci neuronowej

In [130]:
# tworzymy pierwsza warstwe sieci
horsepower_normalizer = layers.Normalization(input_shape=[1,], axis=None)

In [131]:
# przekazujemy metodzie tylko jedną zmienna objasniajaća
horsepower_normalizer.adapt(horsepower)

Budowa modelu Keras

In [ ]:
horsepower_model = tf.keras.Sequential([
    horsepower_normalizer,
    layers.Dense(units=1)# domyślie przyjmuje, że funkcja aktywacji jest lioniowa
])

horsepower_model.summary()

Po zbudowaniu modelu należy go skompilować.

Na tym etapie kluczowy jest dobór hiperparametrów modelu:
*   'loss'
*   'optimizer'



In [133]:
horsepower_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), # 'Adam' jeden z stochastycznych mtod gradientowych (Stochastic Gradient Descent)
    loss='mean_absolute_error')

Proces uczenia modelu

In [ ]:
%%time

# zmienna history będzie zawierała pełną charakterystyką zrealizowanego procesu uczenia
history = horsepower_model.fit(
    train_features['Moc'],
    train_labels,
    epochs=100,
    verbose=1,
    validation_split = 0.2) #Oblicza wyniki walidacji na 20% danych treningowych

Wizualizacja procesu uczenia przy użyciu statystyk zapisanych w obiekcie `history`.

In [ ]:
hist = pd.DataFrame(history.history)
hist.head(5)

In [ ]:
hist['epoch'] = history.epoch
hist.head()

In [137]:
#definiujemy procedure
def plot_loss(history):

  #--------------------------
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #--------------------------

  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
horsepower_model.summary()

In [ ]:
# wyświetlimy wyznaczone współczynniki wagowe modelu

wagi = horsepower_model.get_weights()
wagi

Zachowanie rezultatów na poźniejsze porównania

In [141]:
# tworzymy słownik dla analizy porównawczej - oceny wskażników ewaluacyjnych (stopnia dokładności predykcyjnej modelu)
test_results = {}

# !!! ewaluacja modelu
# obliczamy wskaznik 'loss' dla danych testowych w opariu o metodę "evaluate"
test_results['horsepower_model'] = horsepower_model.evaluate(
    test_features['Moc'], # dane dla jednej zmiennej objaśnijacej
    test_labels, # dla danych !!!! TESTOWYCH
    verbose=1) # etykiety dla danych tesowych

3/3 [==============================] - 0s 3ms/step - loss: 3.6384


In [ ]:
#wynik (loss) dla danych testowych , dla modelu z jednym neuronem
test_results

Ponieważ jest to regresja dla pojedynczej zmiennej, łatwo jest wyświetlić prognozy modelu jako funkcję danych wejściowych (zmiennej 'Moc').

In [143]:
# generujemy sztuczne dane  testowe  do przeprowadzenia weryfikacji wyników predykcji
x = tf.linspace(5.0, 150, 251)

In [144]:
# dokonujemy predykcji MPG na podstwie danych testowych
y = horsepower_model.predict(x)

8/8 [==============================] - 0s 1ms/step


In [ ]:
y[:5]

In [145]:
# definiuję procedurę do tworzenia wykresu
# wykres złożony  z danych treningowych  oraz wyników predykcji dla sztucznych danych "testowych"
def plot_horsepower(x, y):

# ----------------------------------------------
  plt.scatter(train_features['Moc'], train_labels, label='Data') # dane rzeczywiste

  plt.plot(x, y, color='r', label='Predictions') # wyniki predykcji
# ----------------------------------------------

  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

In [ ]:
plot_horsepower(x, y)

### =====================================================================
### Regresja liniowa z wieloma zmiennymi objaśniajacymi

Ponownie budujemy dwuetapowy model sekwencyjny Keras z pierwszą warstwą będącą normalizatorem (tf.keras.layers.Normalization(axis=-1)), który zdefiniowaliśmy wcześniej **[~~ komórka [42]** i dostosowali  do całego zbioru danych (z wszystkimi zmiennymi objaśniajacymi)


In [147]:
# definujemy taki sam jak poprzednio model (z taką samą topologię)
linear_model = tf.keras.Sequential([
    normalizer, # obiekt normalize zawiera tym razem  znormalizowane wartości wszystkich atrybutów(zmiennych objasniających)
    layers.Dense(units=1) # neuron z lioniową funkcją aktywacji
])

Kompilujemy model i przechodzimy do jego uczenia

In [148]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
# ustawiam takie same wartości hiperparametrów jak poprzednio
%%time
history = linear_model.fit(
    train_features,# wszzystkie zmienne objasniajace
    train_labels,
    epochs=100,
    verbose=1,
    validation_split = 0.2)

Wykorzystując wszystkie zmienne objaśniajace uzyskujemy znacznie niższy błąd treningowy i walidacyjny niż w poprzedniom modelu (`horsepower_model`)

In [ ]:
linear_model.summary()

In [ ]:
plot_loss(history)

Zapisujemy wyniki ewaluacji modelu dla danych testowych

In [ ]:
# przeprowadzamy ewaluację modelu
test_results['linear_model'] = linear_model.evaluate(
    test_features, test_labels, verbose=1)

I porównajmy wskażniki ewaluacyjne dla dwóch modeli  z jedną zmienna objaśniającą i wieloma zmiennymi objasniającymi

In [ ]:
print(test_results)

**Pierwsze wnioski ???**

## Regresja za pomocą sieci głębokich [deep neural network (DNN)]


Zaimplementujemy  modele DNN z pojedynczym i wieloma wejściami (uwzględniając jedną  i wszystkie zmienne objaśniające).

Kod jest zasadniczo taki sam, z wyjątkiem tego, że model jest rozszerzony o niektóre "ukryte" warstwy nieliniowe.

Modele te będą zawierać kilka warstw więcej niż model liniowy:



*   Warstwa normalizacji, jak poprzednio (z 'horsepower_normalizer' dla modelu z jednym wejściem i 'normalizer' dla modelu z wieloma wejściami).

*   Dwie ukryte, nieliniowe warstwy Dense z nieliniowością funkcji aktywacji ReLU (relu).
*   Liniowa warstwa Dense z pojedynczym wyjściem.

Oba modele będą korzystać z tej samej procedury uczenia, więc metoda kompilacji jest zawarta w poniższej funkcji build_and_compile_model.

In [154]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)# funkcja aktywacji liniowa
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

### Regresja przy użyciu DNN i pojedynczego wejścia (jedna zmienna objaśniająca )

Zbudujemy model DNN  tylko z `'Moc`` jako danymi wejściowymi i `horsepower_normalizer` (zdefiniowanym wcześniej) jako warstwą normalizacji

In [155]:
dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

Model ten ma o wiele więcej parametrów do wytrenowania niż modele liniowe

In [ ]:
# !!!! przedyskutować topologię  sieci
dnn_horsepower_model.summary()

Proces trenowania modelu (za pomocą metody  Keras `Model.fit`)

In [ ]:
%%time
history = dnn_horsepower_model.fit(
    train_features['Moc'],
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

Ten model radzi sobie nieco lepiej niż jednowejściowy  model liniowy `horsepower_model`

In [ ]:
plot_loss(history)

Jeśli wygenerujemy wykres predykcji ajko funkcji zmiennej 'Moc' łatwo zauważyć jak ten model wykorzystuje nieliniowość zapewnianą przez ukryte warstwy:

In [ ]:
x = tf.linspace(50.0, 250, 251)
y = dnn_horsepower_model.predict(x)

In [ ]:
plot_horsepower(x, y)

Zapisujemy rezultat ewaluacji dla tego modelu (na danych testowych)
Na końcu notebok- a przedstwimy porównaie tego wskażnika dla wszystkich modeli

In [161]:
test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
    test_features['Moc'], test_labels,
    verbose=0)

In [ ]:
test_results

### Regresja przy wykorzystaniu modelu DNN i wszystkich zmiennych objaśniajacych

Powtórzymy poprzedni proces, używając wszystkich dane wejściowe. Wydajność modelu nieznacznie poprawia się na zestawie danych walidacyjnych.

In [ ]:
# podajemy procedurze pełny normalizator (z wszystkimi zmiennymi objasniającymi)
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=100)

In [ ]:
plot_loss(history)

Zapisujemy rezultat ewaluacji tego modelu (dla danych testowych)

In [166]:
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)

## Porównanie wydajności zbudowanych modeli

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

### Predykcja

Dokonamy teraz predykcji za pomocą `dnn_model` **na zbiorze testowym** używając Keras `Model.predict`

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')

#------------------------------------------------------
plt.scatter(test_labels, test_predictions)
#------------------------------------------------------

plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')

#------------------------------------------------------
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


Im bliższe skupienie punktów będących wynikami predykcji wokół prostej (rzeczywista wartosc =  przewidywna wartosc) tym lepiej

Wygląda na to, że model przewiduje dość dobrze.

Teraz należy sprawdzić rozkład błędów:

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')